In [1]:
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch import autograd
from torch.autograd import Variable
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [ ]:
class Discriminator(nn.Module):
    def _init_(self,
                 input_chanel: list,
                 channel_sizes: list,
                 kernel_sizes: list,
                 
                 ann_layer_sizes: list,
                 dropout: float
                 ) -> None:
        
        self.input_channel: int = input_chanel
        self.channel_sizes: list = channel_sizes
        self.kernel_sizes: list  = kernel_sizes

        self.ann_layer_sizes: list = ann_layer_sizes
        self.dropout: float = dropout 

        super()._init_()
        
        # you may define your model in here but i prefer to define it in the forward function
        self.model = nn.Sequential(
            nn.Conv2d(self.input_channel, self.channel_sizes[0], self.kernel_sizes[0], padding= "same"), # 6-256-256 -> 16-256-256
            #nn.Conv2d(self.channel_sizes[0], self.channel_sizes[0], self.kernel_sizes[0], padding= "same"), # 16-256-256 -> 16-256-256
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.MaxPool2d(kernel_size= 2, stride= 2), # 16-256-256 -> 16-128-128

            nn.Conv2d(self.channel_sizes[0], self.channel_sizes[1], self.kernel_sizes[1], padding= "same"), # 16-128-128 -> 32-128-128
            #nn.Conv2d(self.channel_sizes[1], self.channel_sizes[1], self.kernel_sizes[1], padding= "same"), # 32-128-128 -> 32-128-128
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.MaxPool2d(kernel_size= 2, stride= 2), # 32-128-128 - 32-64-64

            nn.Conv2d(self.channel_sizes[1], self.channel_sizes[2], self.kernel_sizes[2], padding= "same"), # 32-64-64 -> 64-64-64
            #nn.Conv2d(self.channel_sizes[2], self.channel_sizes[2], self.kernel_sizes[2], padding= "same"), # 64-64-64 -> 64-64-64
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.MaxPool2d(kernel_size= 2, stride= 2), # 64-64-64 -> 64-32-32
            
            nn.Conv2d(self.channel_sizes[2], self.channel_sizes[3], self.kernel_sizes[3], padding= "same"), # 64-32-32 -> 128-32-32
            #nn.Conv2d(self.channel_sizes[3], self.channel_sizes[3], self.kernel_sizes[3], padding= "same"), # 128-32-32 -> 128-32-32
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.MaxPool2d(kernel_size= 2, stride= 2), # 128-32-32 -> 128-16-16

            nn.Flatten(),

            nn.Linear(self.ann_layer_sizes[0], self.ann_layer_sizes[1]),
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.Linear(self.ann_layer_sizes[1], self.ann_layer_sizes[2]),    
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.Linear(self.ann_layer_sizes[2], self.ann_layer_sizes[3]),
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.Linear(self.ann_layer_sizes[3], 1),
            nn.Sigmoid()

        ) # ??? 
    
    def forward(self, real_image, generated_image):
        x = torch.cat([real_image, generated_image], 2)
        out = self.model(x)
        return out

In [ ]:
def generator_train_step(batch_size: int, instance_shape: list, discriminator, generator, generator_optimizer, criterion, label_image):

    generator_optimizer.zero_grad()

    noise = torch.randn(batch_size, instance_shape[0], instance_shape[1]).to(device)
    
    altered_image = generator(noise, label_image)

    validity = discriminator.forward(altered_image, label_image) # ?? constructor of discriminator is wrong if you want to get result you should use your forward function
    

    #torch.nn.BCELoss(weight=None, size_average=None, reduce=None, reduction='mean')[SOURCE]
    #Creates a criterion that measures the Binary Cross Entropy between the target and the input probabilities:
    g_loss = criterion(validity, torch.ones(batch_size).to(device)) 
    
    
    g_loss.backward()

    generator_optimizer.step()

    return g_loss.data

In [ ]:
for epoch in range(epochs):

    print("Starting epoch {}...".format(epoch + 1))

    for red_apples, green_apples in zip(red_data_loader, green_data_loader):
        
        red_apples = red_apples.to(device) 
        green_apples = green_apples.to(device)

        generator.train()
        # you encountiring the Attribute Error because you are passing instance to BCE which is not a tensor 
        # you should pass the output of the discriminator to the BCE loss function
        d_loss = discriminator_train_step(batch_size, discriminator,
                                          generator, discriminator_optimizer,
                                          criterion, red_apples, green_apples, (256, 256))
        
        g_loss = generator_train_step(batch_size, (256, 256), discriminator, 
                                      generator, generator_optimizer, criterion)
    
    generator.eval()

    print("generator loss: {}, discriminator loss: {}".format(g_loss, d_loss))